# **Lab 2: Matrix factorization**
**Kristoffer Almroth**

In collaboration with
**Timas Ljungdahl**

# **Abstract**

Second lab in the course DD2363 Methods in Scientific Computing. This lab is about matrix factorization and solving the equation Ax=b.

# **Set up environment**

Dependencies needed for running the code.

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D
from random import randrange

import unittest

# **Introduction**

One important function of matrices is to be able to solve the equation Ax=b. This equation can be solved by a variety of different methods, imcluding direct methods and iterative methods. In this lab a direct method will be implemented, based on QR-factorization and the methods to invert orthogonal and upper triangular matrices. QR-factorization is also used to approximate Schur factorization. In addition, a function for sparse matrix-vector product is implemented.

# **Methods**

This section is based on the theory from the third lecture and the lecture notes. Some of the algorithms are based on the pseudo code described in the lecture notes. All code is tested using various different test cases, including the test cases given in the lab requirements and some additional tests. 

Since floating point precision were used in the tests, there could be rounding errors.  The test methods of the library [numpy](https://docs.scipy.org/doc/numpy/reference/routines.testing.html) is used to test equality to a certain precision, here 1e-5.


## Sparse matrix-vector product

The input matrix is on the form Compressed Row Storage, which allows quick matrix multiplication for sparse matrices.

In [0]:
def SparseMatrixVectorProduct(val, col_idx, row_ptr, rows, cols, x):

  # Check dimensions
  assert cols == x.size

  b = np.zeros(rows)
  for i in range(0, rows):
    for j in range(row_ptr[i], row_ptr[i+1]):
      b[i] += val[j]*x[col_idx[j]]
  return b

In [66]:
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,1000):
      row = randrange(100) + 10
      col = randrange(100) + 10
      m = np.random.rand(row,col)
      v = np.random.rand(col)
      
      # Set most of the elements in the matrix to 0, making it a sparse matrix
      for j in range(0, row):
        for k in range(0, col-8):
          m[j][randrange(col)] = 0

      # Create a CRS matrix
      val = []
      col_idx = []
      row_ptr = [0]
      rowNum = 0
      for j in range(0, row):
        for k in range(0, col):
          if m[j][k] != 0:
            val.append(m[j][k])
            col_idx.append(k)
            rowNum += 1

        # Valid since no row is entirely 0
        row_ptr.append(rowNum)

      # Comparing against normal matrix-vector product
      np.testing.assert_allclose(SparseMatrixVectorProduct(val, col_idx, row_ptr, row, col, v), m.dot(v), rtol=1e-5, atol=0)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 9.317s

OK


## QR factorization

QR-factorization is used to easily be able to invert a matrix. A matrix A is factorized into two matrices Q and R, where Q is an orthogonal matrix and R is an upper triangular matrix. Orthogonal matrices can be inverted by transposing them and upper triangular matrices can be inverted by backwards substitution.

We get the orthogonal matrix by the following formula:

$v_j = a_{:j} - \sum_{i=1}^{j-1} (a_{:j}, q_i)q_i$

$q_j = v_j/||v_j||$

The upper triangular matrix is calculated by the formula:

$r_{ij} = (a_{:j},q_i)$

$r_{jj} = ||v_j||$

In [0]:
def QRFactorization(A):

  n = A.shape[0]
  V = np.copy(A)
  R = np.zeros(shape=(n,n))
  Q = np.zeros(shape=(n,n))

  for i in range(0, n):
    R[i,i] = np.linalg.norm(V[:,i])
    Q[:,i] = V[:,i]/R[i,i]
    for j in range(i+1, n):
      R[i,j] = np.dot(Q[:,i], V[:,j])
      V[:,j] = V[:,j] - R[i,j]*Q[:,i]

  return Q, R;

In [68]:
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,100):
      row = randrange(30) + 10
      m = np.random.rand(row,row)
      
      (Q,R) = QRFactorization(m)

      # Test upper triangular
      np.testing.assert_allclose(R, np.triu(R))

      # || Q^TQ-I ||_F = sqrt(n)
      np.testing.assert_almost_equal(np.linalg.norm(np.matmul(np.transpose(Q), np.linalg.inv(Q))), np.sqrt(row), decimal=5)

      # || QR-A ||_F = 0
      np.testing.assert_almost_equal(np.linalg.norm(np.subtract(np.matmul(Q, R), m)), 0, decimal=5)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.618s

OK


## Direct solver for equation $Ax=b$

To solve the equation $Ax=b$ we first use QR-factorization to get two matrices that are easy to invert. 

$QRx=b \Rightarrow x=R^{-1}Q^{-1}b $

An orthogonal matrix can be inverted by transposing it:

$Q^{-1} = Q^T$

$Q_{ij} \Rightarrow Q^T = Q_{ji}$

An upper triangular matrix can be inverted by using backwards substitution

### Backwards substitution

$x_j = b_i - \sum_{j=i+1}^{n} (u_{ij}x_j) / u_{ii}$

In [0]:
def BackwardsSubstitution(U, b):

  n = b.size
  x = np.zeros(n)

  x[n-1] = b[n-1]/U[n-1, n-1]
  for i in range(n-2, -1, -1):
    sum = 0
    for j in range(i+1, n):
      sum += U[i,j]*x[j]
    x[i] = (b[i] - sum) / U[i,i]
  return x

In [70]:
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,100):
      row = randrange(30) + 10
      A = np.random.rand(row,row)
      (Q,R) = QRFactorization(A)
      x = np.random.rand(row)
      b = R.dot(x)

      # Test x = R^(-1)b
      np.testing.assert_allclose(BackwardsSubstitution(R,b), x, rtol=1e-5, atol=0)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.335s

OK


###$Ax=b$

In [0]:
def DirectSolver(A, b):

  (Q, R) = QRFactorization(A)
  return BackwardsSubstitution(R, np.transpose(Q).dot(b))

In [72]:
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,100):
      row = randrange(30) + 10
      A = np.random.rand(row,row)
      x = np.random.rand(row)
      b = A.dot(x)
      x2 = DirectSolver(A,b)

      # x = A^(-1)b
      np.testing.assert_allclose(x2, x, rtol=1e-5, atol=0)

      # ||Ax-b|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(A.dot(x2)-b), 0, decimal=5)

      # ||x-x2|| = 0
      np.testing.assert_almost_equal(np.linalg.norm(x-x2), 0, decimal=5)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.349s

OK


## QR eigenvalue algorithm

This algorithm is an iterative approximation of the Schur factorization of real symmetric matrices. 

$A=UTU^*$ where U has the eigenvectors as column vectors and T is a upper triangular matrix with the eigenvalues on the diagonal.

The iteration algorithm is slow to run, it is thus only tested by small matrices of size 5, with 2000 iterations which gives a precision of over 5 decimals. 

In [0]:
def QREigenvalue(A):

  n = A.shape[0]
  U = np.identity(n)

  for i in range(0, 2000):
    (Q,R) = QRFactorization(A)
    A = np.matmul(R,Q)
    U = np.matmul(U,Q)

  return A, U;

In [74]:
class Test(unittest.TestCase):

  def testRandom(self):
    for i in range(0,10):

      n = 5
      I = np.identity(n)

      # A is a real symmetric matrix
      A = np.random.rand(n,n)
      for i in range(0, n):
        for j in range(0, n):
          A[i,j] = A[j,i]

      (A2,U) = QREigenvalue(A)

      for i in range(0, n):

        # || Av_i - lambda_i v_i || = 0       
        np.testing.assert_almost_equal(np.linalg.norm(A.dot(U[:,i]) - A2[i,i]*U[:,i]), 0, decimal=5)
        
        # det(A - lambda_i I) = 0
        np.testing.assert_almost_equal(np.linalg.det(A - A2[i,i]*I), 0, decimal=5)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 2.818s

OK


# **Results**

The resulting functions as well as their test cases and explanation can be seen under Methods.

# **Discussion**

This lab was more challenging than the last one, but also more interesting. Implementing methods to solve the equation $Ax=b$ is nothing I have done before, which was fun. Using the QR-factorization for both solving the equation $Ax=b$ and to approximate the eigenvalues and the eigenvectors shows the importance of the factorization method.